In [2]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns


/tmp/ipykernel_249053/4141160823.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
train_data = pd.read_csv('data/train.csv.zip', compression='zip')
test_data = pd.read_csv('data/test.csv.zip', compression='zip')
sample_submission_data = pd.read_csv("data/sample_submission.csv")

label_encoder = LabelEncoder()
train_data['Target'] = label_encoder.fit_transform(train_data['Target'])
train_data = train_data.drop(['id'], axis=1)
test_data = test_data.drop(['id'], axis=1)

train_data.drop_duplicates(inplace = True)
train_data.dropna(inplace = True)


In [4]:
train_data.drop(["Previous qualification", "Nacionality", "Mother's qualification", "Father's qualification", "Mother's occupation", 
         "Father's occupation", "Educational special needs", "International", "Curricular units 1st sem (credited)", 
         "Curricular units 1st sem (without evaluations)", "Curricular units 2nd sem (credited)", "Curricular units 2nd sem (grade)", 
         "Curricular units 2nd sem (without evaluations)", "Unemployment rate", "Inflation rate"], axis=1, inplace=True)
test_data.drop(["Previous qualification", "Nacionality", "Mother's qualification", "Father's qualification", "Mother's occupation", 
         "Father's occupation", "Educational special needs", "International", "Curricular units 1st sem (credited)", 
         "Curricular units 1st sem (without evaluations)", "Curricular units 2nd sem (credited)", "Curricular units 2nd sem (grade)", 
         "Curricular units 2nd sem (without evaluations)", "Unemployment rate", "Inflation rate"], axis=1, inplace=True)

In [5]:
data_train_targets = pd.DataFrame(train_data["Target"])
data_train_features = train_data.drop(columns = ["Target"])
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# using pca to extract the features


pca = PCA(n_components=10)
scaler = StandardScaler()

pipeline = Pipeline([('scaler', scaler), ('pca', pca)])
X_scaled_pca = pipeline.fit_transform(data_train_features)



In [6]:
X_train,X_test,y_train,y_test = train_test_split( data_train_features, data_train_targets, test_size=0.10)
y_train=y_train.values.ravel()

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier


estimators = [
    ('svr', make_pipeline(StandardScaler(),
                           LinearSVC(random_state=42))),
    ('mlp',make_pipeline(StandardScaler(), MLPClassifier(alpha=1, max_iter=100)))
    
 ]
clf = StackingClassifier(
     estimators=estimators, final_estimator=LogisticRegression(random_state=42)
 )


clf.fit(X_train, y_train).score(X_test, y_test)




0.8178254051228437

In [8]:
svr_clf =make_pipeline(StandardScaler(),
                           LinearSVC(random_state=42))
svr_clf.fit(X_train, y_train).score(X_test, y_test)

0.8050182958703607

In [9]:
mlp_clf =make_pipeline(StandardScaler(), MLPClassifier(alpha=1, max_iter=100))
mlp_clf.fit(X_train, y_train).score(X_test, y_test)

0.8188708834291688

#Boosting demo



In [10]:
from sklearn.ensemble import AdaBoostClassifier
boost_clf = AdaBoostClassifier(n_estimators=40, random_state=0)
boost_clf.fit(X_train, y_train).score(X_test, y_test)


/home/tiffanybao/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


0.8170412963930999

In [11]:
from sklearn.tree import DecisionTreeClassifier
dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train, y_train).score(X_test, y_test)

0.7349712493465761